In [125]:
import numpy as np 
import matplotlib.pyplot as plt
import cv2
import os
from pathlib import Path
import natsort
import os

In [126]:
# def make_video(image_path, output_video_path):
#     # Generate video:
#     images = natsort.natsorted(os.listdir(image_path))
#     # Path(output_video_path).parent.mkdir(parents=True, exist_ok=True)
#     print(images[0])
#     H, W, C = cv2.imread(os.path.join(image_path,images[0])).shape
#     # if Path(output_video_path).exists():
#     #     Path(output_video_path).unlink()
    
#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#     video = cv2.VideoWriter(f'{output_video_path}\\inference', fourcc, 24, (W, H))
#     for id, image_name in enumerate(images):
#         # image = cv2.imread(str(Path(image_path) / image_name))
#         image = cv2.imread(os.path.join(image_path,image_name))
#         video.write(image)
#     video.release()


In [127]:

def make_video(image_path, output_video_path, output_filename="inference.mp4", fps=24):
    # Ensure output directory exists
    Path(output_video_path).mkdir(parents=True, exist_ok=True)

    # Get sorted list of image files
    images = natsort.natsorted(os.listdir(image_path))

    if not images:
        raise ValueError("No images found in the provided directory.")

    # Read first image to get dimensions
    first_image_path = os.path.join(image_path, images[0])
    first_image = cv2.imread(first_image_path)
    if first_image is None:
        raise ValueError(f"Could not read the first image: {first_image_path}")
    H, W, _ = first_image.shape

    # Create VideoWriter object
    output_file = os.path.join(output_video_path, output_filename)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_file, fourcc, fps, (W, H))

    for image_name in images:
        image_path_full = os.path.join(image_path, image_name)
        image = cv2.imread(image_path_full)
        if image is None:
            print(f"Warning: Skipping unreadable file {image_path_full}")
            continue
        video.write(image)

    video.release()
    print(f"Video saved to {output_file}")


In [128]:
def get_path_and_poses(img_id , cropped=True):
    img_inf = f'color\\{img_id}.png'
    img_uncrop = f'D:\\emjmd_mir\\norway\\thesis\\thesis_ws\\deepurl_docker\\deep_underwater_localization-1.0\\oneposeplus_inference\\uxo\\500lb\\color_full\\{img_id}.jpg'
    pred_path = f'D:\\emjmd_mir\\norway\\thesis\\thesis_ws\\deepurl_docker\\deep_underwater_localization-1.0\\oneposeplus_inference\\uxo\\500lb\\predicted_pose\\{img_id}.txt' 
    gt_path = f'./oneposeplus_inference/small_set/poses_ba/{img_id}.txt'
    k_path = f'intrin_ba\\{img_id}.txt'

    #Original Intrinsic 
    fx,fy,cx,cy = 1400.00, 1400, 1920, 1080

    K = np.array([[fx,0,cx],
                    [0,fy,cy],
                        [0,0,1]])

    pose = np.loadtxt(pred_path)
    # pose_gt = np.loadtxt(gt_path)
    # K_cropped = np.loadtxt(k_path)
    if cropped :
        return img_inf, pose # pose_gt, K_cropped
    else:
        return img_uncrop, pose,  K 

In [129]:
def compute_projection(points_3D, transformation, internal_calibration):
    projections_2d = np.zeros((2, points_3D.shape[1]), dtype='float32')
    camera_projection = (internal_calibration.dot(transformation)).dot(points_3D)
    # camera_projection = (internal_calibration@(transformation))@(points_3D)
    projections_2d[0, :] = camera_projection[0, :] / camera_projection[2, :]
    projections_2d[1, :] = camera_projection[1, :] / camera_projection[2, :]
    return projections_2d

In [130]:
def draw_demo_img_corners(img, projectpts, color = (0, 255, 0), nV=8, thickness = 5):

    vertices = []
    for i in range(nV):
        x = projectpts[i][0]
        y = projectpts[i][1]
        coordinates = (int(x), int(y))
        vertices.append(coordinates)
        cv2.circle(img, coordinates, 2, color, -1)

    # print(vertices)
    cv2.line(img, vertices[0], vertices[1], color, thickness = thickness)
    cv2.line(img, vertices[0], vertices[2], color, thickness = thickness)
    cv2.line(img, vertices[0], vertices[4], color, thickness = thickness)
    cv2.line(img, vertices[1], vertices[5], color, thickness =  thickness)
    cv2.line(img, vertices[1], vertices[3], color, thickness = thickness)
    cv2.line(img, vertices[2], vertices[3], color, thickness = thickness)
    cv2.line(img, vertices[2], vertices[6], color, thickness = thickness)
    cv2.line(img, vertices[3], vertices[7], color, thickness = thickness)
    cv2.line(img, vertices[4], vertices[5], color, thickness = thickness)
    cv2.line(img, vertices[4], vertices[6], color, thickness = thickness)
    cv2.line(img, vertices[5], vertices[7], color, thickness =  thickness)
    cv2.line(img, vertices[6], vertices[7], color, thickness = thickness)

    return img

In [131]:
# # Docking Station
# min_x,min_y,min_z = -310.211914,-626.390503,-617.810913
# max_x,max_y,max_z = 310.211914,626.390503,617.810913
# corners = np.array([[min_x, min_y, min_z],
#                     [min_x, min_y, max_z],
#                     [min_x, max_y, min_z],
#                     [min_x, max_y, max_z],
#                     [max_x, min_y, min_z],
#                     [max_x, min_y, max_z],
#                     [max_x, max_y, min_z],
#                     [max_x, max_y, max_z]])
# corners = np.concatenate((np.transpose(corners), np.ones((1, 8))), axis=0)
# corners

In [132]:
# #Inc_shell
# min_x,min_y,min_z = -2.1459435,-0.6427165,-0.699005
# max_x,max_y,max_z =  2.1459435,  0.6427165,  0.699005
# corners = np.array([[min_x, min_y, min_z],
#                     [min_x, min_y, max_z],
#                     [min_x, max_y, min_z],
#                     [min_x, max_y, max_z],
#                     [max_x, min_y, min_z],
#                     [max_x, min_y, max_z],
#                     [max_x, max_y, min_z],
#                     [max_x, max_y, max_z]])
# corners = np.concatenate((np.transpose(corners), np.ones((1, 8))), axis=0)
# corners

In [133]:
#500lb
min_x,min_y,min_z = -1.627172, -0.4971785, -0.5440315
max_x,max_y,max_z =  1.627172,  0.4971785,  0.5440315
corners = np.array([[min_x, min_y, min_z],
                    [min_x, min_y, max_z],
                    [min_x, max_y, min_z],
                    [min_x, max_y, max_z],
                    [max_x, min_y, min_z],
                    [max_x, min_y, max_z],
                    [max_x, max_y, min_z],
                    [max_x, max_y, max_z]])
corners = np.concatenate((np.transpose(corners), np.ones((1, 8))), axis=0)
corners

array([[-1.627172 , -1.627172 , -1.627172 , -1.627172 ,  1.627172 ,
         1.627172 ,  1.627172 ,  1.627172 ],
       [-0.4971785, -0.4971785,  0.4971785,  0.4971785, -0.4971785,
        -0.4971785,  0.4971785,  0.4971785],
       [-0.5440315,  0.5440315, -0.5440315,  0.5440315, -0.5440315,
         0.5440315, -0.5440315,  0.5440315],
       [ 1.       ,  1.       ,  1.       ,  1.       ,  1.       ,
         1.       ,  1.       ,  1.       ]])

In [134]:
# pose_gt = np.loadtxt(pose_file)
# R = rotation_180_z@np.array(pose_gt)[:3,:3]
# T = np.array(pose_gt)[:3,3:]* 1000
# pose  = np.concatenate((R, T), 1)
# projections = compute_projection(corners,pose,K)
# print(projections)
# corners2D_pr = np.transpose(projections) #add .flatten() for saving annotations
# flat = corners2D_pr.flatten().astype(float)
# print(corners2D_pr)
# print(flat)
# print(projections.flatten())
# # print(projections[1:,1:])

In [135]:
# img_ori = cv2.imread(image_file)
# # print(img_ori)
# img_ori = draw_demo_img_corners(img_ori,flat.reshape(-1, 2), (0, 0, 255), nV=8)
# # cv2.imwrite("data\\deepurl_docking_station\\ex_box3.jpg", img_ori)
# cv2.imshow('Image', img_ori)
# k = cv2.waitKey(0) & 0XFF
# cv2.destroyAllWindows()

Project point and save video

In [136]:
pose_path = 'D:\\emjmd_mir\\norway\\thesis\\thesis_ws\\deepurl_docker\\deep_underwater_localization-1.0\\oneposeplus_inference\\uxo\\500lb\\predicted_pose' 


In [137]:
#Plot boxes and save image
# i = 0
pose_files = os.listdir(pose_path)
num_images = len(pose_files)
print(num_images)
for idx in range(num_images):
    # image_width, image_height = 2048,1536
    # # YOLOv8 detection output: [class_id, x_center, y_center, width, height, confidence]
    # yolo_detection = np.loadtxt(os.path.join(yolo_path,f'{'.'.join(image.split('.')[0:2])}.txt'))  # Example

    # # Convert YOLO format to (xmin, ymin, xmax, ymax)
    # x_center, y_center, width, height = yolo_detection[1:5]
    # xmin = int((x_center - width / 2) * image_width)
    # ymin = int((y_center - height / 2) * image_height)
    # xmax = int((x_center + width / 2) * image_width)
    # ymax = int((y_center + height / 2) * image_height)
    # bbox = [xmin, ymin, xmax, ymax]
    # bbox = list(map(float, bbox))
    # print(bbox)
    filename, pose, K  = get_path_and_poses(idx , cropped=False)
    R = np.array(pose)[:3,:3]
    T = np.array(pose)[:3,3:] #*1000 for docking station
    pose  = np.concatenate((R, T), 1)
    projections = compute_projection(corners,pose,K)
    keypoints = np.transpose(projections)

    img_ori = cv2.imread(filename)
    # print(img_ori)
    # print(img_ori)
    try: 
        img_ori = draw_demo_img_corners(img_ori, keypoints, (255, 0,0 ), nV=8)
    except:
        print("something went wrong:",idx)
    cv2.imwrite(f"D:\\emjmd_mir\\norway\\thesis\\thesis_ws\\deepurl_docker\\deep_underwater_localization-1.0\\oneposeplus_inference\\uxo\\500lb\\pose_plot\\{idx}_box3d.jpg", img_ori)

    # line = [i, path_to_img, image_width, image_height, 0]
    # line.extend(bbox)  # xmin, ymin, xmax, ymax
    # line.extend(keypoints)  # x1, y1, ..., x8, y8
    # label_line = " ".join(map(str, line)) + "\n"
    # print(label_line)
    # f.write(label_line)
    
#  with open(save_file, "w") as f:

300


In [138]:
frame_path=f"D:\\emjmd_mir\\norway\\thesis\\thesis_ws\\deepurl_docker\\deep_underwater_localization-1.0\\oneposeplus_inference\\uxo\\500lb\\pose_plot"
vdo_save_path =f"D:\\emjmd_mir\\norway\\thesis\\thesis_ws\\deepurl_docker\\deep_underwater_localization-1.0\\oneposeplus_inference\\uxo\\500lb\\inference_video" 
make_video(frame_path, vdo_save_path,output_filename="inference3.mp4",fps=30)
#build video 

Video saved to D:\emjmd_mir\norway\thesis\thesis_ws\deepurl_docker\deep_underwater_localization-1.0\oneposeplus_inference\uxo\500lb\inference_video\inference3.mp4
